In [ ]:
# import os
# import hashlib
# from PIL import Image
# import imagehash

# def calculate_image_hash(image_path):
#     # 使用 imagehash 计算图片的哈希值
#     image = Image.open(image_path)
#     return imagehash.average_hash(image)

# def delete_duplicate_images(folder_path):
#     seen_hashes = set()
#     for root, dirs, files in os.walk(folder_path):
#         for file in files:
#             file_path = os.path.join(root, file)
#             if file.lower().endswith(('png', 'jpg', 'jpeg', 'gif', 'bmp')):
#                 # 计算图片的哈希值
#                 image_hash = calculate_image_hash(file_path)
#                 if image_hash in seen_hashes:
#                     print(f"删除重复图片: {file_path}")
#                     os.remove(file_path)  # 删除重复图片
#                 else:
#                     seen_hashes.add(image_hash)

# folder_path = './album_cover'  # 请替换为你的文件夹路径
# delete_duplicate_images(folder_path)


删除重复图片: ./album_cover/image_218.jpg
删除重复图片: ./album_cover/image_11.jpg
删除重复图片: ./album_cover/image_197.jpg
删除重复图片: ./album_cover/image_133.jpg
删除重复图片: ./album_cover/image_89.jpg
删除重复图片: ./album_cover/image_292.jpg
删除重复图片: ./album_cover/image_279.jpg
删除重复图片: ./album_cover/image_250.jpg
删除重复图片: ./album_cover/image_278.jpg
删除重复图片: ./album_cover/image_63.jpg
删除重复图片: ./album_cover/image_126.jpg
删除重复图片: ./album_cover/image_130.jpg
删除重复图片: ./album_cover/image_290.jpg
删除重复图片: ./album_cover/image_9.jpg
删除重复图片: ./album_cover/image_131.jpg
删除重复图片: ./album_cover/image_121.jpg
删除重复图片: ./album_cover/image_58.jpg
删除重复图片: ./album_cover/image_70.jpg
删除重复图片: ./album_cover/image_294.jpg
删除重复图片: ./album_cover/image_243.jpg
删除重复图片: ./album_cover/image_120.jpg
删除重复图片: ./album_cover/image_136.jpg
删除重复图片: ./album_cover/image_98.jpg
删除重复图片: ./album_cover/image_283.jpg
删除重复图片: ./album_cover/image_99.jpg
删除重复图片: ./album_cover/image_137.jpg
删除重复图片: ./album_cover/image_112.jpg
删除重复图片: ./album_cover/image_106.jpg
删

In [23]:
import os
from PIL import Image
import numpy as np
import colorsys
import glob

def get_average_color(image_path):
    # 打开图片
    img = Image.open(image_path)
    compress_ratio = 10
    img = img.resize((img.width // compress_ratio, img.height // compress_ratio))  # 降低图片分辨率加快计算
    
    # 转换为RGB模式并转换为NumPy数组
    img_rgb = np.array(img.convert('RGB'))
    
    # 计算所有像素的平均值
    avg_color = np.mean(img_rgb, axis=(0, 1))
    
    return avg_color

def rgb_to_hsv(rgb):
    # 将RGB转换为HSV格式
    r, g, b = rgb / 255.0  # 将RGB值归一化到0-1范围
    h, s, v = colorsys.rgb_to_hsv(r, g, b)
    return h, s, v

def sort_by_hue(image_folder):
    # 获取文件夹中的所有图片
    image_paths = glob.glob(os.path.join(image_folder, "*.jpg")) + glob.glob(os.path.join(image_folder, "*.png"))
    
    colors = []
    
    # 提取每张图片的主色并转换为HSV
    for img_path in image_paths:
        avg_rgb = get_average_color(img_path)
        hue, sat, val = rgb_to_hsv(avg_rgb)
        colors.append((img_path, hue))
    
    # 按照色调值进行排序
    colors.sort(key=lambda x: x[1])  # 按照色调hue排序
    
    return colors

def create_collage(sorted_images, images_per_row=10, spacing=10, target_size=(120, 120)):
    # 计算每行的图片数
    num_images = len(sorted_images)
    rows = (num_images + images_per_row - 1) // images_per_row  # 向上取整
    
    # 获取单张图片的尺寸
    img_width, img_height = target_size  # 目标大小作为缩放后的尺寸
    
    # 计算拼贴图的总宽度和总高度
    total_width = min(images_per_row, num_images) * img_width + (min(images_per_row, num_images) - 1) * spacing
    total_height = rows * img_height + (rows - 1) * spacing
    
    # 创建空白画布
    collage = Image.new('RGB', (total_width, total_height), (0, 0, 0))
    
    # 贴图到画布
    x_offset = 0
    y_offset = 0
    for i, (img_path, _) in enumerate(sorted_images):
        img = Image.open(img_path)
        
        # 缩小图片
        img = img.resize(target_size)
        
        collage.paste(img, (x_offset, y_offset))
        
        # 更新偏移量
        if (i + 1) % images_per_row == 0:  # 每行结束时换行
            x_offset = 0
            y_offset += img_height + spacing
        else:
            x_offset += img_width + spacing
    
    return collage


# 设置文件夹路径
image_folder = "album_cover"

# 获取排序后的图片文件路径
sorted_images = sort_by_hue(image_folder)

# 创建拼贴图，设置间隔为10像素
collage = create_collage(sorted_images, spacing=0)

# 保存合并后的图片
collage.save('color_wheel_collage.png')

# 显示合并后的图片
collage.show()


In [25]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import colorsys

# 获取图片文件夹中的图片
def get_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            images.append(img_path)
    return images

# 获取图片的亮度和色相
def get_brightness_and_hue(image_path):
    img = Image.open(image_path)
    img = img.convert('RGB')
    pixels = np.array(img)
    
    # 计算亮度（平均亮度）
    brightness = np.mean(pixels[..., :3]) / 255.0
    
    # 计算色相
    avg_color = np.mean(pixels[..., :3], axis=(0, 1)) / 255.0
    hue = colorsys.rgb_to_hsv(avg_color[0], avg_color[1], avg_color[2])[0]
    
    return brightness, hue

# 按亮度和色相对图片进行排序
def sort_images(images):
    images_with_properties = []
    for img_path in images:
        brightness, hue = get_brightness_and_hue(img_path)
        images_with_properties.append((img_path, brightness, hue))
    
    # 按照亮度升序，然后按色相升序排序
    images_with_properties.sort(key=lambda x: (x[1], x[2]))
    
    return [img[0] for img in images_with_properties]

# 将图片缩小并放置在一个平面上
def create_image_grid(images, output_path, scale_factor=0.2, spacing=10, images_per_row=5):
    img_objects = []
    
    # 读取和缩小图片
    for img_path in images:
        img = Image.open(img_path)
        img = img.resize((int(img.width * scale_factor), int(img.height * scale_factor)))
        img_objects.append(img)
    
    # 计算画布的大小
    grid_width = max(img.width for img in img_objects) * images_per_row + spacing * (images_per_row - 1)
    grid_height = max(img.height for img in img_objects) * (len(img_objects) // images_per_row + 1) + spacing * (len(img_objects) // images_per_row)
    
    # 创建一个空白画布
    grid_img = Image.new('RGB', (grid_width, grid_height), (255, 255, 255))
    
    x_offset = 0
    y_offset = 0
    
    for i, img in enumerate(img_objects):
        grid_img.paste(img, (x_offset, y_offset))
        
        x_offset += img.width + spacing
        if (i + 1) % images_per_row == 0:
            x_offset = 0
            y_offset += img.height + spacing
    
    # 保存输出图像
    grid_img.save(output_path)

# 主程序
def main():
    folder_path = "album_cover"
    output_path = "brightness.jpg"     # 输出图像路径
    
    images = get_images_from_folder(folder_path)
    sorted_images = sort_images(images)
    create_image_grid(sorted_images, output_path, scale_factor=0.2, spacing=0, images_per_row=10)
    print(f"输出图像已保存至 {output_path}")

# 执行主程序
if __name__ == '__main__':
    main()


输出图像已保存至 brightness.jpg


In [12]:
import os
from PIL import Image, ImageDraw
import numpy as np
import colorsys

# 获取图片文件夹中的图片
def get_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            images.append(img_path)
    return images

# 获取图片的亮度和色相
def get_brightness_and_hue(image_path):
    img = Image.open(image_path)
    img = img.convert('RGB')
    pixels = np.array(img)
    
    # 计算亮度（平均亮度）
    brightness = np.mean(pixels[..., :3]) / 255.0
    
    # 计算色相
    avg_color = np.mean(pixels[..., :3], axis=(0, 1)) / 255.0
    hue = colorsys.rgb_to_hsv(avg_color[0], avg_color[1], avg_color[2])[0]
    
    return brightness, hue

# 按亮度和色相对图片进行排序
def get_brightness_and_hue_for_images(images):
    images_with_properties = []
    for img_path in images:
        brightness, hue = get_brightness_and_hue(img_path)
        images_with_properties.append((img_path, brightness, hue))
    
    return images_with_properties

# 在亮度-色相二维空间上排列图片
def create_image_grid(images, output_path, scale_factor=0.2, canvas_size=(2000, 2000), density_factor=1.0):
    """
    根据图片的亮度和色相值，生成并保存一张图片网格。

    :param images: 图片文件路径列表
    :param output_path: 输出图片的保存路径
    :param scale_factor: 图片缩放比例
    :param canvas_size: 画布大小（宽, 高）
    :param density_factor: 控制图像密集程度，值越大，图片之间的间隔越大
    """
    img_objects = []
    image_properties = get_brightness_and_hue_for_images(images)

    # 计算亮度和色相值
    brightness_hue_coords = [(prop[1], prop[2]) for prop in image_properties]
    brightness_values = [b for b, h in brightness_hue_coords]
    hue_values = [h for b, h in brightness_hue_coords]

    min_brightness, max_brightness = min(brightness_values), max(brightness_values)
    min_hue, max_hue = min(hue_values), max(hue_values)

    def normalize(value, min_val, max_val, canvas_dim):
        return int(((value - min_val) / (max_val - min_val)) * (canvas_dim - 1))

    # 创建空白画布
    canvas = Image.new("RGB", canvas_size, "black")
    draw = ImageDraw.Draw(canvas)

    occupied_positions = set()  # 用于存储已放置的图像位置，防止重叠

    # 处理每张图片
    for img_path, (brightness, hue) in zip(images, brightness_hue_coords):
        img = Image.open(img_path).convert("RGBA")
        img = img.resize((int(img.width * scale_factor), int(img.height * scale_factor)))
        img_width, img_height = img.size

        # 计算放置坐标
        x = normalize(brightness, min_brightness, max_brightness, canvas_size[0])
        y = normalize(hue, min_hue, max_hue, canvas_size[1])

        # 计算偏移，防止超出边界
        x = min(max(x, img_width // 2), canvas_size[0] - img_width // 2)
        y = min(max(y, img_height // 2), canvas_size[1] - img_height // 2)

        # 根据密集度调整坐标
        spacing = int(min(img_width, img_height) * density_factor)  # 计算间距
        while (x, y) in occupied_positions:  # 避免重叠
            x += spacing
            y += spacing
            x = min(max(x, img_width // 2), canvas_size[0] - img_width // 2)
            y = min(max(y, img_height // 2), canvas_size[1] - img_height // 2)

        # 记录占用的位置
        occupied_positions.add((x, y))

        # 贴图
        canvas.paste(img, (x - img_width // 2, y - img_height // 2), img)

    # 保存最终的图像
    canvas.save(output_path, "PNG")

# 主程序
def main():
    folder_path = "album_cover"
    output_path = "hue-brightness.jpg"     # 输出图像路径
    
    images = get_images_from_folder(folder_path)
    create_image_grid(images, output_path, scale_factor=0.3)
    print(f"输出图像已保存至 {output_path}")

# 执行主程序
if __name__ == '__main__':
    main()


输出图像已保存至 hue-brightness.jpg


In [1]:
import os
from PIL import Image, ImageDraw
import numpy as np
import colorsys

# 获取图片文件夹中的图片
def get_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder_path, filename)
            images.append(img_path)
    return images

# 获取图片的亮度和色相
def get_brightness_and_hue(image_path):
    img = Image.open(image_path)
    img = img.convert('RGB')
    pixels = np.array(img)
    
    # 计算亮度（平均亮度）
    brightness = np.mean(pixels[..., :3]) / 255.0
    
    # 计算色相
    avg_color = np.mean(pixels[..., :3], axis=(0, 1)) / 255.0
    hue = colorsys.rgb_to_hsv(avg_color[0], avg_color[1], avg_color[2])[0]
    
    return brightness, hue

# 按亮度和色相对图片进行排序
def get_brightness_and_hue_for_images(images):
    images_with_properties = []
    for img_path in images:
        brightness, hue = get_brightness_and_hue(img_path)
        images_with_properties.append((img_path, brightness, hue))
    
    return images_with_properties

# 在亮度-色相二维空间上排列图片
import numpy as np
from PIL import Image, ImageDraw
from continuous_rounded_corner import round_corner

def create_image_grid(images, output_path, base_scale=0.2, canvas_size=(2000, 2000), density_factor=1.0, bins=10):
    """
    根据图片的亮度和色相值，生成并保存一张图片网格，并动态调整图片缩放比例。

    :param images: 图片文件路径列表
    :param output_path: 输出图片的保存路径
    :param base_scale: 图片的基础缩放比例
    :param canvas_size: 画布大小（宽, 高）
    :param density_factor: 控制图像密集程度，值越大，图片之间的间隔越大
    :param bins: 直方图分箱的数量
    """
    img_objects = []
    image_properties = get_brightness_and_hue_for_images(images)

    # 计算亮度和色相值
    brightness_hue_coords = [(prop[1], prop[2]) for prop in image_properties]
    brightness_values = np.array([b for b, h in brightness_hue_coords])
    hue_values = np.array([h for b, h in brightness_hue_coords])

    min_brightness, max_brightness = brightness_values.min(), brightness_values.max()
    min_hue, max_hue = hue_values.min(), hue_values.max()

    def normalize(value, min_val, max_val, canvas_dim):
        return int(((value - min_val) / (max_val - min_val)) * (canvas_dim - 1))

    # 计算 2D 直方图，统计每个区域的图片数量
    hist, x_edges, y_edges = np.histogram2d(brightness_values, hue_values, bins=bins)

    # 归一化图片密度
    hist = hist.T  # 需要转置以匹配 (hue, brightness) 形式
    hist_density = (hist + 1) ** 0.7  # 避免除零，加1防止过小
    max_density = hist_density.max()

    # 创建空白画布
    canvas = Image.new("RGB", canvas_size, "black")
    draw = ImageDraw.Draw(canvas)

    occupied_positions = set()  # 用于存储已放置的图像位置，防止重叠

    # 处理每张图片
    for img_path, (brightness, hue) in zip(images, brightness_hue_coords):
        img = Image.open(img_path).convert("RGBA")

        # 找到图片在直方图中的索引
        bin_x = min(np.digitize([brightness], x_edges)[0] - 1, bins - 1)
        bin_y = min(np.digitize([hue], y_edges)[0] - 1, bins - 1)

        # 计算缩放比例
        density = hist_density[bin_y, bin_x]
        scale_factor = base_scale / (density / max_density) # 缩放比例与密度成反比
        print(scale_factor)
        scale_factor = np.clip(scale_factor, 0.02, 0.4)  # 限制缩放范围

        img = img.resize((int(img.width * scale_factor), int(img.height * scale_factor)))
        img = round_corner(img)
        
        img_width, img_height = img.size

        # 计算放置坐标
        x = normalize(brightness, min_brightness, max_brightness, canvas_size[0])
        y = normalize(hue, min_hue, max_hue, canvas_size[1])

        # 计算偏移，防止超出边界
        x = min(max(x, img_width // 2), canvas_size[0] - img_width // 2)
        y = min(max(y, img_height // 2), canvas_size[1] - img_height // 2)

        # 根据密集度调整坐标
        spacing = int(min(img_width, img_height) * density_factor)  # 计算间距
        while (x, y) in occupied_positions:  # 避免重叠
            x += spacing
            y += spacing
            x = min(max(x, img_width // 2), canvas_size[0] - img_width // 2)
            y = min(max(y, img_height // 2), canvas_size[1] - img_height // 2)

        # 记录占用的位置
        occupied_positions.add((x, y))

        # 贴图
        canvas.paste(img, (x - img_width // 2, y - img_height // 2), img)

    # 保存最终的图像
    canvas.save(output_path, "PNG")


# 主程序
def main():
    folder_path = "album_cover"
    output_path = "hue-brightness.jpg"     # 输出图像路径
    
    images = get_images_from_folder(folder_path)
    # create_image_grid(images, output_path, base_scale=0.3 * 0.25, canvas_size=(2000, 2000))
    create_image_grid(images, output_path, base_scale=0.3 * 0.4, canvas_size=(4000, 2250))
    print(f"输出图像已保存至 {output_path}")

# 执行主程序
if __name__ == '__main__':
    main()


0.1685693459559093
0.2342408253527242
0.1850857421271501
0.3349320551938721
0.44485717100527017
0.1552286341110701
0.2738417104097813
0.1685693459559093
0.2342408253527242
0.3349320551938721
0.44485717100527017
0.3349320551938721
0.2738417104097813
0.12
0.1685693459559093
0.1850857421271501
0.44485717100527017
0.3349320551938721
0.13488594939028795
0.13488594939028795
0.2738417104097813
0.2342408253527242
0.12
0.44485717100527017
0.2738417104097813
0.2738417104097813
0.1685693459559093
0.1552286341110701
0.1850857421271501
0.2342408253527242
0.3349320551938721
0.1685693459559093
0.13488594939028795
0.13488594939028795
0.20617486430103343
0.2738417104097813
0.44485717100527017
0.13488594939028795
0.2738417104097813
0.2738417104097813
0.3349320551938721
0.13488594939028795
0.13488594939028795
0.2738417104097813
0.20617486430103343
0.1850857421271501
0.2738417104097813
0.44485717100527017
0.2738417104097813
0.3349320551938721
0.2738417104097813
0.12
0.44485717100527017
0.20617486430103343